# PDE1-1: 1D Allen-Cahn Equation

$$
\frac{∂u(x, t)}{∂t}+0.0001\frac{∂^2u(x, t)}{∂x^2}+5u(x, t)^3-5u(x, t)=0
$$

with $x \in [-1, 1]$, and $x \in [0, 1]$. The boundary condition defined as:
$$
u(x, 0)=x^2 cos(\pi x)
$$
$$
u(t, -1)=u(t, 1) \\
$$
$$
u_{x}(x, -1)=u_{x}(x, 1)
$$

In [ ]:
import numpy as np
import warnings
import scipy.io
from pyDOE import lhs
warnings.filterwarnings('ignore')

# Data size on the solution
data = scipy.io.loadmat('AC.mat')

t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
exact = data['uu']
exact_u = np.real(exact)

x_grid, t_grid = np.meshgrid(x, t)

# Preparing the x and t together as an input for predictions in one single array, as data_all
data_all = np.hstack((x_grid.flatten()[:, None], t_grid.flatten()[:, None]))

# Preparing the exact solution in similar position correspond with each combination input x and t
data_exact = exact_u.flatten()[:, None]

# Determine how much points for training in both boundary condition and PDE itself
N_b = 50
N_u = 1500

# Bounday condition and exact value when u(x, 0) (left)
xx1 = np.hstack((x_grid[0:1, :].T, t_grid[0:1, :].T))
idx1 = np.random.choice(xx1.shape[0], N_b, replace=False)
xx1_b = xx1[idx1, :]
uu1 = exact_u[:, 0:1]
uu1_b = uu1[idx1, :]

# Bounday condition and exact value when u(x, 1) (right)
xx2 = np.hstack((x_grid[0:1, :].T, t_grid[-1:, :].T))
idx2 = np.random.choice(xx2.shape[0], N_b, replace=False)
xx2_b = xx2[idx2, :]
uu2 = exact_u[:, -1:]
uu2_b = uu2[idx2, :]

# Bounday condition and exact value when u(-1, t) (bottom)
xx3 = np.hstack((x_grid[:, 0:1], t_grid[:, 0:1]))
idx3 = np.random.choice(xx3.shape[0], N_b, replace=False)
xx3_b = xx3[idx3, :]
uu3 = exact_u[0:1, :].T
uu3_b = uu3[idx3, :]

# Bounday condition and exact value when u(1, t) (top)
xx4 = np.hstack((x_grid[:, -1:], t_grid[:, -1:]))
idx4 = np.random.choice(xx4.shape[0], N_b, replace=False)
xx4_b = xx4[idx4, :]
uu4 = exact_u[-1:, :].T
uu4_b = uu4[idx4, :]

# Stacking all boundary condition the exact value in single variable
data_b_all = np.vstack([xx1, xx2, xx3, xx4])
exact_b_all = np.vstack([uu1, uu2, uu3, uu4])

xx = [xx1, xx2, xx3, xx4]
uu = [uu1, uu2, uu3, uu4]

idx_b = [np.random.choice(x.shape[0], N_b, replace=False) for x in xx]

data_b_train = np.vstack([xx1_b, xx2_b, xx3_b, xx4_b])
exact_b_train = np.vstack([uu1_b, uu2_b, uu3_b, uu4_b])

# Domain bounds (lowerbounds upperbounds) [x, t], which are here ([0, 0]) and ([1, 1])
lb = data_all.min(axis=0)
ub = data_all.max(axis=0)

# Preparing the training data inside PDE domain
data_u_train = lb + (ub-lb)*lhs(2, N_u)

idx = np.random.choice(data_u_train.shape[0], N_u, replace=False)

exact_u_train = data_exact[idx, :]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RectBivariateSpline

fig, axs = plt.subplots(1, 2, figsize=[12, 4])

axs[0].set_title("Collocation and initial/boundary training points", fontsize=15, fontweight ='bold')
axs[0].set_xlabel('t', fontsize=12, fontweight ='bold')
axs[0].set_ylabel('x', fontsize=12, fontweight ='bold')
axs[0].vlines(x = 0, ymin = -1, ymax = 1, color='black', linewidth = 1)
axs[0].vlines(x = 1, ymin = -1, ymax = 1, color='black', linewidth = 1)
axs[0].hlines(y = -1, xmin = 0, xmax = 1, color='black', linewidth = 1)
axs[0].hlines(y = 1, xmin = 0, xmax = 1, color='black', linewidth = 1)
dot = axs[0].scatter(data_u_train[:,1], data_u_train[:,0], s=5, marker='o', label = 'PDE points', clip_on = False)
cross1 = axs[0].scatter(data_b_train[:,1], data_b_train[:,0], s=10, marker='x', label = 'Boundary points', 
                  linewidth = 2, clip_on = False)
cross2 = axs[0].scatter(data_b_train[:50][:,1], data_b_train[:50][:,0], s=10, marker='x', color='red', label = 'Initial points', 
                  linewidth = 2, clip_on = False)
axs[0].get_xaxis().set_visible(True)
axs[0].get_yaxis().set_visible(True)

axs[0].legend(handles=[dot, cross2, cross1], labels=['Collocation points', 'Initial points', 'Boundary points'], loc='upper center', 
           bbox_to_anchor=(0.5, -0.1), ncol=3, frameon=False)

cax = axs[1].contourf(t_grid, x_grid, exact_u.T, 50, cmap='jet', origin='lower')
axs[1].set_title('Ground-truth solution', fontsize=15, fontweight='bold')
#ax1.get_xaxis().set_visible(False)
axs[1].set_xlabel('t', fontsize=12, fontweight='bold')
axs[1].set_ylabel('x', fontsize=12, fontweight='bold')

# Add a colorbar
cbar = fig.colorbar(cax)

plt.show()

In [ ]:
import os
import time
import math
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.activations import tanh

class FLANNLayer(tf.keras.layers.Layer):
    def __init__(self, output_dim, degree, activation=None, minval=-0.05, maxval=0.05, **kwargs):
        super(FLANNLayer, self).__init__(**kwargs)
        self.output_dim = output_dim
        self.degree = degree
        self.function_type = function_type
        self.activation = tf.keras.activations.get(activation)
        self.minval = minval
        self.maxval = maxval

    def build(self, input_shape):
        self.feature_size = (self.degree + 1) * input_shape[1]
        self.kernel = self.add_weight(
            name='kernel',
            shape=(self.feature_size, self.output_dim),
            initializer=tf.keras.initializers.HeNormal(),
            trainable=True
        )

    def apply_activation(self, x):
        if self.activation is not None:
            return self.activation(x)
        return x

    def legendre_polynomials(self, x, degree):
        batch_size, input_dim = tf.shape(x)[0], tf.shape(x)[1]
        P = [self.apply_activation(tf.ones((batch_size, input_dim), dtype=x.dtype)), self.apply_activation(x)]
        for n in range(2, degree + 1):
            Pn = ((2.0 * n - 1.0) * x * P[n - 1] - (n - 1.0) * P[n - 2]) / n
            P.append(self.apply_activation(Pn))
        return P

    def call(self, inputs):
        F = self.legendre_polynomials(inputs, self.degree)
        outputs = tf.concat(F, axis=1)
        outputs = tf.matmul(outputs, self.kernel)
        if self.activation is not None:
            outputs = self.apply_activation(outputs)
        return outputs

keras.utils.get_custom_objects()["FLANNLayer"] = FLANNLayer

In [ ]:
import os
import time
import math
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.activations import tanh

class PINN:
    def __init__(self, data_u_train, exact_u_train, data_b_train, exact_b_train, optimizer):
        self.data_u_train = data_u_train
        self.exact_u_train = exact_u_train
        self.data_b_train = data_b_train
        self.exact_b_train = exact_b_train
        self.optimizer = optimizer
        self.build_model()
        
    def build_model(self):
        # Define input layer
        inputs = tf.keras.Input(shape=self.data_u_train.shape[1:])
        
        dense1 = Dense(50, activation='tanh')(inputs)
        dense2 = Dense(50, activation='tanh')(dense1)
        dense3 = Dense(50, activation='tanh')(dense2)
        dense4 = Dense(50, activation='tanh')(dense3)
        flann = FLANNLayer(output_dim=50, degree=5, function_type='legendre', activation='tanh')(dense4)
        outputs = Dense(1)(flann)

        # Define the model
        self.model_nn = Model(inputs=inputs, outputs=outputs)
        self.model_nn.compile(optimizer=self.optimizer, loss=self.__loss)
        
        # Initialize the history list
        self.history = {"loss": [], "val_loss": []}
        
    def fit_model(self, epochs):
        earlystopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        checpoint_cb = keras.callbacks.ModelCheckpoint('pde11_lel_pinn.h5', monitor='val_loss', save_best_only=True)
        history_callback = self.model_nn.fit(self.data_u_train, self.exact_u_train, epochs=epochs,
                          callbacks=[checpoint_cb], verbose=1, validation_split=0.25)
        self.history["loss"].append(history_callback.history["loss"])
        self.history["val_loss"].append(history_callback.history["val_loss"])

    def u_model(self):
        x_f = tf.convert_to_tensor(self.data_u_train[:, 0:1], dtype=tf.float32)
        t_f = tf.convert_to_tensor(self.data_u_train[:, 1:2], dtype=tf.float32)
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x_f)
            tape.watch(t_f)
            with tf.GradientTape(persistent=True) as gtape:
                gtape.watch(x_f)
                gtape.watch(t_f)
                input = tf.stack([x_f[:, 0], t_f[:, 0]], axis=1)
                u = self.model_nn(input)
            u_x = gtape.gradient(u, x_f)
            u_t = gtape.gradient(u, t_f)
            del gtape
        u_xx = tape.gradient(u_x, x_f)
        del tape
        
        return u_t - 0.0001*u_xx + 5.0*(u)**3 - 5.0*u
    
    def u_x_model(self, data_b_train):
        x_b = tf.convert_to_tensor(data_b_train[:, 0:1], dtype=tf.float32)
        t_b = tf.convert_to_tensor(data_b_train[:, 1:2], dtype=tf.float32)
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x_b)
            tape.watch(t_b)
            X_b = tf.stack([x_b[:, 0], t_b[:, 0]], axis=1)
            u = self.model_nn(X_b)
        u_x = tape.gradient(u, x_b)
        del tape
        return u, u_x

    def __loss(self, data_u_train, exact_u_train):
        data_b_0 = tf.convert_to_tensor(self.data_b_train[:50], dtype=tf.float32) #init
        data_b_1 = tf.convert_to_tensor(self.data_b_train[50:100], dtype=tf.float32) #right
        data_b_2 = tf.convert_to_tensor(self.data_b_train[100:150], dtype=tf.float32) #bottom
        data_b_3 = tf.convert_to_tensor(self.data_b_train[150:200], dtype=tf.float32) #top
        
        exact_b_0 = tf.convert_to_tensor(self.exact_b_train[:50], dtype=tf.float32) #exact init
        exact_b_1 = tf.convert_to_tensor(self.exact_b_train[50:100], dtype=tf.float32) #exact right
        exact_b_2 = tf.convert_to_tensor(self.exact_b_train[100:150], dtype=tf.float32) #exact bottom
        exact_b_3 = tf.convert_to_tensor(self.exact_b_train[150:200], dtype=tf.float32) #exact top
        
        init_pred, _ = self.u_x_model(data_b_0)
        right_pred, _ = self.u_x_model(data_b_1)
        uub_pred, uub_x_pred = self.u_x_model(data_b_3)
        ulb_pred, ulb_x_pred = self.u_x_model(data_b_2)
        u_pred = self.u_model()
    
        loss_0 = tf.reduce_mean(tf.square(init_pred - (data_b_0[:, 0:1]**2)*tf.math.cos(np.pi*data_b_0[:, 0:1])) + \
                               tf.square(right_pred - exact_b_1))
        loss_b = tf.reduce_mean(tf.square(ulb_pred - exact_b_2) + tf.square(ulb_pred - exact_b_3) + \
                                tf.square(ulb_x_pred - uub_x_pred))
        loss_u = tf.reduce_mean(tf.square(u_pred))

        return loss_0 + loss_b + loss_u
    
    def predict(self, x):
        prediction = self.model_nn.predict(x)
        return prediction
    
    keras.utils.get_custom_objects()["__loss"] = __loss

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

pde_pinn = PINN(data_u_train, exact_u_train, data_b_train, exact_b_train, optimizer)
pde_pinn.model_nn.summary()

In [ ]:
import time

start = time.perf_counter()
for i in range(1):
    print('cycle:------------------------------------', i+1)
    pde_pinn.fit_model(epochs=2000)
    pde_pinn.model_nn.load_weights('pde11_lel_pinn.h5')
        
elapsed = time.perf_counter() - start
print('Elapsed %.3f seconds.' % elapsed)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 1, figsize=[5, 3])
axs.set_facecolor('lightgrey')
axs.grid(which='major', color='black', linestyle='--', linewidth=0.5)
axs.plot(pde_pinn.history["loss"][0], label='LEL-PINN Train Loss', color='black')
axs.plot(pde_pinn.history["val_loss"][0], label='LEL-PINN Test Loss', color='orange')
axs.set_yscale('log')
axs.set_xlim([-50, 2050])
axs.set_ylim([5e-6, 5e+0])
axs.set_xlabel('Iter', fontsize=12, fontweight ='bold')
axs.set_ylabel('Loss value', fontsize=12, fontweight ='bold', x=0)
plt.legend()
plt.show()

In [ ]:
from sklearn import metrics

lel_pinn_prediction = pde_pinn.predict(data_all)

In [ ]:
from numpy import format_float_scientific

index = pde_pinn.history["val_loss"][0].index(min(pde_pinn.history["val_loss"][0]))

print(format_float_scientific(pde_pinn.history["loss"][0][index], exp_digits=2, precision=3))
print(format_float_scientific(pde_pinn.history["val_loss"][0][index], exp_digits=2, precision=3))
print(len(pde_pinn.history["val_loss"][0]))
print('%.2f'%(elapsed/len(pde_pinn.history["val_loss"][0])))
print('%.2f'%elapsed)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = plt.figure(figsize=[8, 5])  # Increased height to fit additional subplots

lel_pinn_value_pred = lel_pinn_prediction.reshape((t.shape[0], x.shape[0]))

# Create a grid with 2 rows and 3 columns; the first row will span all three columns for the main plot
gs = GridSpec(2, 3, height_ratios=[3, 1], hspace=0.4)  # Adjust `height_ratios` as needed

line = np.linspace(x.min(), x.max(), 2)[:,None]

# Adjust main plot to fully align with bottom plots
axs_main = fig.add_axes([0.12, 0.4, 0.82, 0.55])  # [left, bottom, width, height]
divider = make_axes_locatable(axs_main)

# Create contour plot
axs_main.set_title('$u(x, t)$ LEL-PINN Prediction', fontsize=12, fontweight='bold')
cax = axs_main.contourf(t_grid, x_grid, lel_pinn_value_pred, 50, cmap='jet', origin='lower')
axs_main.plot(t[40] * np.ones((2, 1)), line, 'k--', linewidth=1)
axs_main.plot(t[100] * np.ones((2, 1)), line, 'k--', linewidth=1)
axs_main.plot(t[160] * np.ones((2, 1)), line, 'k--', linewidth=1)
axs_main.set_xlabel('t', fontsize = 12, fontweight='bold')
axs_main.set_ylabel('x', fontsize = 12, fontweight='bold')

# Create an axis for the colorbar outside the plot
cbar_ax = divider.append_axes("right", size="3%", pad=0.1)
colorbar = plt.colorbar(cax, cax=cbar_ax)
colorbar.set_label('$u(x, t)$')  # Label for colorbar

lel_pinn_u_pred = griddata(data_all, lel_pinn_prediction.flatten(), (x_grid, t_grid), method='cubic')

# Small subplots on the second row
axs1 = fig.add_subplot(gs[1, 0])
axs1.set_facecolor('lightgrey')
axs1.grid(which='major', color='black', linestyle='--', linewidth=0.5)
axs1.plot(x, exact_u.T[40,:], alpha=0.5, ls='-', lw=5,)
axs1.plot(x, lel_pinn_u_pred[40,:], ls='--', lw=1, color='blue')
axs1.set_ylim([-1.25, 1.25])
axs1.set_title('$u(x, t)$ at $t=%.2f$' % (t[40]), fontsize=8)
axs1.set_ylabel('$u(x, t)$', fontsize = 8, fontweight='bold')

axs2 = fig.add_subplot(gs[1, 1])
axs2.set_facecolor('lightgrey')
axs2.grid(which='major', color='black', linestyle='--', linewidth=0.5)
axs2.plot(x, exact_u.T[100,:], alpha=0.5, ls='-', lw=5,)
axs2.plot(x, lel_pinn_u_pred[100,:], ls='--', lw=1, color='blue')
axs2.set_ylim([-1.25, 1.25])
axs2.set_title('$u(x, t)$ at $t=%.2f$' % (t[100]), fontsize=8)
axs2.set_xlabel('x', fontsize = 8, fontweight='bold')

axs3 = fig.add_subplot(gs[1, 2])
axs3.set_facecolor('lightgrey')
axs3.grid(which='major', color='black', linestyle='--', linewidth=0.5)
axs3.plot(x, exact_u.T[160,:], alpha=0.5, ls='-', lw=5, label = 'Ground-truth sol')
axs3.plot(x, lel_pinn_u_pred[160,:], ls='--', lw=1, color='blue', label='LEL-PINN')
axs3.set_ylim([-1.25, 1.25])
axs3.set_title('$u(x, t)$ at $t=%.2f$' % (t[160]), fontsize=8)

axs3.legend(loc='upper center', bbox_to_anchor=(-0.7, -0.4), ncol=4, frameon=False)

plt.show()
